In [14]:
import pandas as pd
import numpy as np
import re
from transformers import BertModel, BertTokenizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
import sklearn.metrics as metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [2]:
dataFile = pd.read_excel("student_info.xlsx",sheet_name='Sheet1')
dataFile

,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,E2,E3,...,E12,E13,E14,E15,E16,E17,E18,E19,E20,E21
0,26373,1101,select name from Customer,0,454,2020-06-15 13:16:04,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,26374,1101,select C.name from Customer as C,0,454,2020-06-15 13:16:26,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27282,1103,select name from customer,0,454,2020-06-16 23:20:56,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28499,1104,SELECT name FROM Customer;,0,454,2020-06-18 22:58:57,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,26942,1107,Select name from customer;,0,454,2020-06-16 11:51:33,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50329,199184,2478,"CREATE TABLE PRODUCER( vineyard VARCHAR(30), ...",8,1220,2023-07-16 20:41:50,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50330,199185,2478,"CREATE TABLE PRODUCER( vineyard VARCHAR(30), ...",0,1220,2023-07-16 20:43:02,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50331,199520,2479,CREATE TABLE ERZEUGER(anbaugebiet varchar(100)...,9,1220,2023-07-25 19:10:32,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50332,199521,2479,CREATE TABLE ERZEUGER(anbaugebiet varchar(100)...,9,1220,2023-07-25 19:11:02,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
selected_rows = dataFile[(dataFile['E1'] == 1) | (dataFile['E2'] == 1) | (dataFile['E3'] == 1) | (dataFile['E4'] == 1) | (dataFile['E5'] == 1) | (dataFile['E6'] == 1) | (dataFile['E7'] == 1) | (dataFile['E8'] == 1) | (dataFile['E9'] == 1) | (dataFile['E10'] == 1) | (dataFile['E11'] == 1) | (dataFile['E12'] == 1) | (dataFile['E13'] == 1) | (dataFile['E14'] == 1) | (dataFile['E15'] == 1) | (dataFile['E16'] == 1) | (dataFile['E17'] == 1) | (dataFile['E18'] == 1) | (dataFile['E19'] == 1) | (dataFile['E20'] == 1) | (dataFile['E21'] == 1)]
df = pd.DataFrame(selected_rows)
df = df.reset_index()
df

,index,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,E2,...,E12,E13,E14,E15,E16,E17,E18,E19,E20,E21
0,70,28417,1216,select * from Customer;,2,454,2020-06-18 18:17:16,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,87,28121,1246,"select cid, name FROM customer GROUP BY name",2,454,2020-06-18 12:01:15,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,98,31056,1277,select * from customer,2,454,2020-06-20 20:42:07,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,157,27291,1103,"SELECT Customer.name, Orders.OidFROM CustomerJ...",2,455,2020-06-16 23:33:31,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,161,27295,1103,select cid from orders,2,455,2020-06-16 23:37:42,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,49437,190815,2283,CREATE TABLE STUDENTS ( student_id int(20) NO...,15,1216,2023-05-05 21:21:19,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1251,49457,191128,2285,"create table STUDENT ( student_id int, f...",16,1216,2023-05-07 11:36:38,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1252,49531,191810,2305,CREATE TABLE STUDENT ( student_id INT PRIMARY...,16,1216,2023-05-08 16:02:28,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1253,49803,199464,2475,CREATE TABLE kneipe ( name VARCHAR(50) PRIMAR...,16,1217,2023-07-25 00:53:16,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [4]:
columns_with_1 = []
for i in range(0, 1255):
    name = df.columns[df.iloc[i].eq(1)].tolist()
    columns_with_1.append(name[0])
df['Severe Error'] = columns_with_1
df

,index,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,E2,...,E13,E14,E15,E16,E17,E18,E19,E20,E21,Severe Error
0,70,28417,1216,select * from Customer;,2,454,2020-06-18 18:17:16,0,0,1,...,0,0,0,0,0,0,0,0,0,E2
1,87,28121,1246,"select cid, name FROM customer GROUP BY name",2,454,2020-06-18 12:01:15,0,0,1,...,0,0,0,0,0,0,0,0,0,E2
2,98,31056,1277,select * from customer,2,454,2020-06-20 20:42:07,0,0,1,...,0,0,0,0,0,0,0,0,0,E2
3,157,27291,1103,"SELECT Customer.name, Orders.OidFROM CustomerJ...",2,455,2020-06-16 23:33:31,0,0,1,...,0,0,0,0,0,0,0,0,0,E2
4,161,27295,1103,select cid from orders,2,455,2020-06-16 23:37:42,0,0,1,...,0,0,0,0,0,0,0,0,0,E2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,49437,190815,2283,CREATE TABLE STUDENTS ( student_id int(20) NO...,15,1216,2023-05-05 21:21:19,0,0,0,...,0,0,1,0,0,0,0,0,0,E15
1251,49457,191128,2285,"create table STUDENT ( student_id int, f...",16,1216,2023-05-07 11:36:38,0,0,0,...,0,0,0,1,0,0,0,0,0,E16
1252,49531,191810,2305,CREATE TABLE STUDENT ( student_id INT PRIMARY...,16,1216,2023-05-08 16:02:28,0,0,0,...,0,0,0,1,0,0,0,0,0,E16
1253,49803,199464,2475,CREATE TABLE kneipe ( name VARCHAR(50) PRIMAR...,16,1217,2023-07-25 00:53:16,0,0,0,...,0,0,0,1,0,0,0,0,0,E16


In [5]:
keyword_mappings = pd.read_excel("SS20-23.xlsx",sheet_name='Mapping')
keyword_mappings

,Exercise-Questionno,Task ID,Task Description,Possible error classes for the task ID,E0,E2,E3,E4,E6,E20,...,E8,E9,E10,E11,E5,E12,E13,E14,E15,E16
0,"8-1(a), 10.1-(a)[23]","454, 786, 972",Get the names of all costumers.,"0,2",Selection of Tables: The from Clause select * ...,Selection of Tables: The from Clause select * ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"8-1(b), 10.1-(b)[23]","455, 787, 973",Get all orders of customer Meier.,"0, 2, 6, 4, 20",select * from WINES where Name = ’La Rose’ or ...,Imprecise Selection /2 Exampleselect * from WI...,NaN,Imprecise Selection /2 Exampleselect * from WI...,Imprecise Selection /2 Exampleselect * from WI...,Imprecise Selection /2 Exampleselect * from WI...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"8-1(c), 10.1-(c)[23]","456, 788, 974",List all products that have not been sold on 1...,"0, 6, 2, 4,20",The where Clause select ...from ... where cond...,select * from WINEMAKER where Name not in (sel...,NaN,"select Name, Color, WINES.Vineyard from WINES,...","select Name, Color, WINES.Vineyard from WINES,...",select * from WINEMAKER where Name not in (sel...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"8-1(d), 10.1-(d)[23]","457, 789, 975",List all products that dealer Schmidt sold to ...,"0,2,4,6",Structure of an SQL Query -- query select proj...,"Query in SQL_x000D_select Name, Color, WINES.V...",NaN,Structure of an SQL Query -- query select proj...,"Query in SQL_x000D_select Name, Color, WINES.V...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8-1(e).1,458,Get all products that dealer Schmidt sold and ...,"0, 6, 2, 20, 4",Structure of an SQL Query -- query select proj...,Set Operations /3 Intersection r1 ∩ r2: yields...,NaN,Set Operations /3 Intersection r1 ∩ r2: yields...,Set Operations /3 Intersection r1 ∩ r2: yields...,Set Operations /3 Intersection r1 ∩ r2: yields...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"8-1(e).2, 10.1-(e)[23]",459,Get all products that dealer Meier sold and cu...,"0,2",The where Clause select ...from ... where cond...,Set Operations Union r1 ∪ r2 of two relations ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"7.1-1(a), 9.1-(a)[23]","464, 796, 961, 1189",Query name and salary of all employees that ea...,"0,2,6",The where Clause select ...from ... where cond...,Range Selection Range selection attrib between...,NaN,NaN,Range Selection Range selection attrib between...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"7.1-1(b), 9.1-(b)[23]","465, 797, 962, 1190",Query name and salary of all employees that do...,"6,0,2",The where Clause select ...from ... where cond...,Range Selection Range selection attrib between...,NaN,NaN,Range Selection Range selection attrib between...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"7.1-1(c), 9.1-(c)[23]","466, 798, 963, 1191",Query all planes that are of type A-340 or TRI...,"0, 2, 6, 21,4",Sorting with order by Notation Example:_x000D_...,Sorting also with calculated attributes (aggre...,NaN,Sorting also with calculated attributes (aggre...,Sorting also with calculated attributes (aggre...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"CREATE, TABLE,SELECT,*,FROM,UPDATE,SET,DELETE,..."
9,"7.1-1(d), 9.1-(d)[23]","467, 799, 964, 1192",Which pilots have a license different than I o...,"6, 0, 2,20",The where Clause select ...from ... where cond...,Synchronized Subqueries Vineyards with 1999 re...,NaN,The where Clause select ...from ... where cond...,"WHERE,<>,<>,IN,NOT IN,WHERE NOT,OR,AND",Synchronized Subqueries Vineyards with 1999 re...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def get_keywords(taskid,errorclass):
    keywords = ''
    for i in range(len(keyword_mappings)):
        try:
            listing = keyword_mappings['Task ID'][i].split(',')
        except:
            listing = []
            l = keyword_mappings['Task ID'][i]
            listing.append(l)
        if(str(taskid) in listing):
            keywords = keyword_mappings[errorclass][i]
    return(keywords)

In [7]:
data = pd.read_csv('slides.csv')

words_to_remove = ['Saake','Database', 'Concepts', 'Last', 'Edited:', 'April', '2022']

def remove_words(text):
    for word in words_to_remove:
        text = text.replace(word, '')
    return text

data['Content'] = data['Content'].apply(lambda x: remove_words(x))

data = data[data['Slide Number'] != 'PDF-2, Pg1']
data = data[data['Slide Number'] != 'PDF-2, Pg16']
data = data[data['Slide Number'] != 'PDF-2, Pg23']
data = data[data['Slide Number'] != 'PDF-2, Pg36']
data = data[data['Slide Number'] != 'PDF-2, Pg41']


data = data[data['Slide Number'] != 'PDF-6, Pg1']
data = data[data['Slide Number'] != 'PDF-6, Pg35']
data = data[data['Slide Number'] != 'PDF-6, Pg40']
data = data[data['Slide Number'] != 'PDF-6, Pg47']
data = data[data['Slide Number'] != 'PDF-6, Pg59']
data = data[data['Slide Number'] != 'PDF-6, Pg77']
data = data[data['Slide Number'] != 'PDF-6, Pg89']
data = data[data['Slide Number'] != 'PDF-6, Pg103']

data = data[data['Slide Number'] != 'PDF-8, Pg1']
data = data[data['Slide Number'] != 'PDF-8, Pg14']
data = data[data['Slide Number'] != 'PDF-8, Pg18']
data = data[data['Slide Number'] != 'PDF-8, Pg27']
data = data[data['Slide Number'] != 'PDF-8, Pg35']
data = data[data['Slide Number'] != 'PDF-8, Pg41']
data = data[data['Slide Number'] != 'PDF-8, Pg49']
data = data[data['Slide Number'] != 'PDF-8, Pg57']
data = data[data['Slide Number'] != 'PDF-8, Pg60']
data = data[data['Slide Number'] != 'PDF-8, Pg65']


In [8]:
def get_bert_encodings(x):
    model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
    embeddings = model.encode(x)
    return embeddings

data['encodings'] = data['Content'].apply(lambda x: get_bert_encodings(x))

In [9]:

recommendation = []
for i in range(len(df)):
    try:
        tid = df['taskid'][i]
        error_class = df['Severe Error'][i]
        keywords = get_keywords(tid, error_class)
        keyword_encodings = get_bert_encodings(keywords)
        keyword_encodings = keyword_encodings.reshape(1, -1)
        data['similarity_score'] = data['encodings'].apply(lambda x: metrics.pairwise.cosine_similarity(x.reshape(1,-1), keyword_encodings)[0][0])
        df_results = data.sort_values(by=['similarity_score'], ascending=False)
        df_results = df_results.reset_index()
        recommendation.append(df_results['Slide Number'][0])
    except:
        recommendation.append("No recommendation")

df['Recommendation'] = recommendation
df


,index,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,E2,...,E14,E15,E16,E17,E18,E19,E20,E21,Severe Error,Recommendation
0,70,28417,1216,select * from Customer;,2,454,2020-06-18 18:17:16,0,0,1,...,0,0,0,0,0,0,0,0,E2,"PDF-6, Pg75"
1,87,28121,1246,"select cid, name FROM customer GROUP BY name",2,454,2020-06-18 12:01:15,0,0,1,...,0,0,0,0,0,0,0,0,E2,"PDF-6, Pg75"
2,98,31056,1277,select * from customer,2,454,2020-06-20 20:42:07,0,0,1,...,0,0,0,0,0,0,0,0,E2,"PDF-6, Pg75"
3,157,27291,1103,"SELECT Customer.name, Orders.OidFROM CustomerJ...",2,455,2020-06-16 23:33:31,0,0,1,...,0,0,0,0,0,0,0,0,E2,"PDF-6, Pg34"
4,161,27295,1103,select cid from orders,2,455,2020-06-16 23:37:42,0,0,1,...,0,0,0,0,0,0,0,0,E2,"PDF-6, Pg34"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,49437,190815,2283,CREATE TABLE STUDENTS ( student_id int(20) NO...,15,1216,2023-05-05 21:21:19,0,0,0,...,0,1,0,0,0,0,0,0,E15,"PDF-8, Pg16"
1251,49457,191128,2285,"create table STUDENT ( student_id int, f...",16,1216,2023-05-07 11:36:38,0,0,0,...,0,0,1,0,0,0,0,0,E16,"PDF-8, Pg16"
1252,49531,191810,2305,CREATE TABLE STUDENT ( student_id INT PRIMARY...,16,1216,2023-05-08 16:02:28,0,0,0,...,0,0,1,0,0,0,0,0,E16,"PDF-8, Pg16"
1253,49803,199464,2475,CREATE TABLE kneipe ( name VARCHAR(50) PRIMAR...,16,1217,2023-07-25 00:53:16,0,0,0,...,0,0,1,0,0,0,0,0,E16,"PDF-8, Pg16"


In [29]:
df.to_csv('recommendation.csv', index=False)

In [10]:
dataFile = pd.read_excel('SS20-23.xlsx',sheet_name='Ground_truth')
dataFile

,TaskID,ErrorClass,Slide Number
0,496,0,"PDF-6, Pg23"
1,496,2,"PDF-2, Pg37"
2,496,6,"PDF-2, Pg39"
3,828,0,"PDF-6, Pg23"
4,828,2,"PDF-2, Pg37"
...,...,...,...
799,791,2,"PDF-1, Pg-1"
800,976,2,"PDF-1, Pg-1"
801,976,6,"PDF-1, Pg-1"
802,977,2,"PDF-1, Pg-1"


In [11]:
#Evaluation

taskid = []
error_class = []
recommendation = []
actual = []
count = 0
taskid = df['taskid']
error_class = df['Severe Error']
recommendation = df['Recommendation']
for i in range(len(taskid)):
    task_id_val = int(taskid[i])
    error_class_val = int(error_class[i][1:])
    s = dataFile.loc[(dataFile['TaskID'] == task_id_val) & (dataFile['ErrorClass'] == error_class_val), 'Slide Number']
    s = str(s)
    pattern = r'PDF[\w\s,-]+(?=\nName)'
    extracted_value = re.search(pattern, s)
    if(extracted_value):
        actual.append(extracted_value.group(0))   

eval_dataset = pd.DataFrame({'TaskID': taskid, 'Error_Class': error_class, 'Recommendation':recommendation, 'Actual':actual})
#eval_dataset.to_csv('eval_dataset.csv')
eval_dataset

,TaskID,Error_Class,Recommendation,Actual
0,454,E2,"PDF-6, Pg75","PDF-6, Pg15"
1,454,E2,"PDF-6, Pg75","PDF-6, Pg15"
2,454,E2,"PDF-6, Pg75","PDF-6, Pg15"
3,455,E2,"PDF-6, Pg34","PDF-6, Pg34"
4,455,E2,"PDF-6, Pg34","PDF-6, Pg34"
...,...,...,...,...
1250,1216,E15,"PDF-8, Pg16","PDF-2, Pg19"
1251,1216,E16,"PDF-8, Pg16","PDF-2, Pg19"
1252,1216,E16,"PDF-8, Pg16","PDF-2, Pg19"
1253,1217,E16,"PDF-8, Pg16","PDF-2, Pg19"


In [17]:

# Extract the actual and predicted labels from the DataFrame
actual_labels = eval_dataset['Actual']
predicted_labels = eval_dataset['Recommendation']


# Calculate precision
precision = precision_score(predicted_labels, actual_labels, average='macro')

# Calculate recall
recall = recall_score(predicted_labels, actual_labels, average='macro')

accuracy = accuracy_score(predicted_labels, actual_labels)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)

Precision: 0.10903967466901747
Recall: 0.27960526315789475
Accuracy: 0.17928286852589642


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
